In [174]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from datetime import datetime
import matplotlib.pyplot as plt
import requests
import requests_cache
import time
import math
from sqlalchemy import create_engine
%matplotlib inline

engine = create_engine('sqlite:///stock.sqlite')
conn = engine.connect()

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36 Edg/85.0.564.51',
          'X-Requested-With': 'XMLHttpRequest'
          }

def get_investing_pair_ids():
    url = 'https://cn.investing.com/stock-screener/Service/SearchStocks'
    df = pd.DataFrame(columns=['pair_id', 'symbol', 'name'])
    page_num = 1
    total_num = 1
    while page_num <= total_num:
        data = {
            'country[]': '37',
            'sector': '2,11,7,10,1,4,9,5,8,3,6,12',
            'industry': '63,85,82,21,10,86,7,78,36,25,4,28,67,5,71,27,61,90,23,68,34,89,43,50,81,41,56,59,69,9,83,29,52,100,58,95,102,94,60,53,38,87,31,6,16,48,55,74,66,35,65,40,99,42,92,98,39,70,32,45,77,20,54,33,24,72,51,30,64,2,96,8,14,22,26,80,15,37,93,13,46,1,79,44,75,91,49,62,88,12,47,84,57,76,17,97,18,19,3,11,101,73',
            'equityType': 'ORD,DRC,Preferred,Unit,ClosedEnd,REIT,ELKS,OpenEnd,Right,ParticipationShare,CapitalSecurity,PerpetualCapitalSecurity,GuaranteeCertificate,IGC,Warrant,SeniorNote,Debenture,ETF,ADR,ETC,ETN',
            'pn': str(page_num),
            'order[col]': 'eq_market_cap',
            'order[dir]': 'd'

        }
        r = requests.post(url, data=data, headers=headers)
        json = r.json()
        
        for stock in json['hits']:
            df = df.append({
                'pair_id': stock['pair_ID'],
                'symbol': stock['stock_symbol'],
                'name': stock['name_trans']
            }, ignore_index=True)
        page_num = int(json['pageNumber']) + 1
        total_num = math.ceil(json['totalCount'] / 50)
    df.to_sql('investing_pair_ids', engine, chunksize=1000, if_exists='replace')
    print('获取', len(df), '条记录')

def get_historical_data(stocks, if_exists='append'):
    fail_stock_list = []
    counter = 0
    url = 'https://cn.investing.com/instruments/HistoricalDataAjax'
    for symbol, pair_id in stocks['pair_id'].items():
        data = {
            'curr_id': pair_id,
            'smlID': '204958',
            'header': '',
            'st_date': st_date,
            'end_date': end_date,
            'interval_sec': 'Daily',
            'sort_col': 'date',
            'sort_ord': 'ASC',
            'action': 'historical_data'
        }
        
        n = 0
        r = requests.post(url, data=data, headers=headers)
        while r.status_code != 200 and n < 10 :
            n += 1
            print('重试第', n, '次')
            if n < 10:
                r = requests.post(url, data=data, headers=headers)
            else:
                fail_stock_list.append(symbol)
                
            
        if r.status_code == 200:
            df = pd.read_html(r.text)[0]
            df['代码'] = str(symbol)
            df['证券简称'] = stocks.loc[symbol]['name']
#             df['日期'] = df['日期'].apply(lambda x: datetime.strptime(x, "%Y年%m月%d日"))
            df[['日期', '收盘', '交易量', '代码', '证券简称']].to_sql('historical_data', engine, chunksize=1000, if_exists=if_exists)
            if_exists = 'append'
            
        counter += 1       
                
        print(str(symbol), stocks.loc[symbol]['name'], '完成..........进度 {:.0%}'.format(counter / len(stocks)), flush=True)
    return fail_stock_list

In [5]:
%%time
get_investing_pair_ids()

获取 4108 条记录
CPU times: user 22.1 s, sys: 298 ms, total: 22.4 s
Wall time: 3min 40s


In [197]:
pair_ids = pd.read_sql_table('investing_pair_ids', conn, index_col='index')
exchange_stocks = pd.read_sql_table('stock_list', conn, index_col='index')
pair_ids.rename(columns={'symbol': '代码'}, inplace=True)
pair_ids[['代码', 'pair_id']] = pair_ids[['代码', 'pair_id']].astype('string')
exchange_stocks['代码'] = exchange_stocks['代码'].astype('string')
stocks = exchange_stocks.set_index('代码').join(pair_ids.set_index('代码'))

In [175]:
%%time
st_date = '2010/01/01'
end_date = time.strftime('%Y/%m/%d', time.localtime())
fail_stocks = get_historical_data(stocks, if_exists='replace')

600000 浦发银行 完成..........进度 0%
600004 白云机场 完成..........进度 0%
600006 东风汽车 完成..........进度 0%
600007 中国国贸 完成..........进度 0%
600008 首创股份 完成..........进度 0%
600009 上海机场 完成..........进度 0%
600010 包钢股份 完成..........进度 0%
600011 华能国际 完成..........进度 0%
600012 皖通高速 完成..........进度 0%
600015 华夏银行 完成..........进度 1%
600016 民生银行 完成..........进度 1%
600017 日照港 完成..........进度 1%
600018 上港集团 完成..........进度 1%
600019 宝钢股份 完成..........进度 1%
600020 中原高速 完成..........进度 1%
600021 上海电力 完成..........进度 1%
600022 山东钢铁 完成..........进度 1%
600023 浙能电力 完成..........进度 1%
600025 华能水电 完成..........进度 1%
600026 中远海能 完成..........进度 1%
600027 华电国际 完成..........进度 1%
600028 中国石化 完成..........进度 1%
600029 南方航空 完成..........进度 1%
600030 中信证券 完成..........进度 1%
600031 三一重工 完成..........进度 1%
600033 福建高速 完成..........进度 1%
600035 楚天高速 完成..........进度 1%
600036 招商银行 完成..........进度 2%
600037 歌华有线 完成..........进度 2%
600038 中直股份 完成..........进度 2%
600039 四川路桥 完成..........进度 2%
600048 保利地产 完成..........进度 2%
600050 中国联通 完成..........进度 2%
600051 宁波联合

KeyError: "['交易量', '日期', '收盘'] not in index"

In [183]:
type(stocks.loc['600145']['pair_id'])

pandas._libs.missing.NAType

In [184]:
len(stocks)

1808

In [202]:
stocks

,公司代码,公司简称,简称,上市日期,证券类型,交易所,pair_id,name
代码,,,,,,,,
600000,600000,浦发银行,浦发银行,1999-11-10,主板A股,上海证券交易所,100276,浦发银行
600004,600004,白云机场,白云机场,2003-04-28,主板A股,上海证券交易所,100277,白云机场
600006,600006,东风汽车,东风汽车,1999-07-27,主板A股,上海证券交易所,100279,东风汽车
600007,600007,中国国贸,中国国贸,1999-03-12,主板A股,上海证券交易所,100280,中国国贸
600008,600008,首创股份,首创股份,2000-04-27,主板A股,上海证券交易所,100281,首创股份
...,...,...,...,...,...,...,...,...
900948,900948,伊泰B股,伊泰B股,1997-08-08,主板B股,上海证券交易所,101200,伊泰B股
900952,600190,锦州港,锦港B股,1999-06-09,主板B股,上海证券交易所,101203,锦港B股
900953,900953,凯马B,凯马B,1998-06-24,主板B股,上海证券交易所,101204,凯马B


In [188]:
len(stocks['pair_id'])

1808

In [194]:
stocks.dropna(inplace=True)

In [198]:
len(stocks)

1808

In [203]:
stocks[stocks['pair_id'].isna()]

,公司代码,公司简称,简称,上市日期,证券类型,交易所,pair_id,name
代码,,,,,,,,
600145,600145,*ST新亿,*ST新亿,1999-09-23,主板A股,上海证券交易所,<NA>,NaN
600651,600651,*ST飞乐,*ST飞乐,1986-09-26,主板A股,上海证券交易所,<NA>,NaN
601187,601187,厦门银行,厦门银行,2020-10-27,主板A股,上海证券交易所,<NA>,NaN
601456,601456,国联证券,国联证券,2020-07-31,主板A股,上海证券交易所,<NA>,NaN
601568,601568,北元集团,北元集团,2020-10-20,主板A股,上海证券交易所,<NA>,NaN
...,...,...,...,...,...,...,...,...
688596,688596,正帆科技,正帆科技,2020-08-20,科创板,上海证券交易所,<NA>,NaN
688777,688777,中控技术,中控技术,2020-11-24,科创板,上海证券交易所,<NA>,NaN
688788,688788,科思科技,科思科技,2020-10-22,科创板,上海证券交易所,<NA>,NaN
